## Vegetation indexes to support the burned area delineation 

Data passed by reference


In [1]:
workflow = dict([('id', 'vegetation-index-ref'),
                ('label', 'Vegetation index by reference'),
                ('doc', 'Vegetation index processor by reference')])


In [2]:
input_reference = dict([('id', 'input_reference'), 
                        ('label', 'STAC item for vegetation index'),
                        ('doc', 'STAC item for vegetation index'),
                        ('value', 'https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_36RTT_20191205_0_L2A'), 
                        ('type', 'string'),
                        ('scatter', 'True')])

In [3]:
aoi = dict([('id', 'aoi'), 
              ('label', 'Area of interest'),
              ('doc', 'Area of interest in WKT'),
              ('value', 'POLYGON((30.358 29.243,30.358 29.545,30.8 29.545,30.8 29.243,30.358 29.243))'), 
              ('type', 'string')])

### Vegetation indexes

NBR = (NIR - SWIR22) / (NIR + SWIR22)
NDVI = (NIR - Red) / (NIR + Red)
NDWI = (NIR - SWIR16) / (NIR + SWIR16)

In [4]:
import os
import sys
import gdal
import numpy as np
import logging
from pystac import *
from shapely.wkt import loads
from time import sleep
from helpers import *
import shutil

set_env()

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
s2_item = S2_stac_item(input_reference['value'])
       
catalog = Catalog(id='catalog_id', 
                  description='catalog_description')


catalog.add_item(s2_item.item)

catalog.normalize_and_save(root_href='instac',
                              catalog_type=CatalogType.SELF_CONTAINED)

In [7]:
cat = Catalog.from_file(os.path.join('instac', 'catalog.json')) 
    
cat.describe()
    

* <Catalog id=catalog_id>
  * <Item id=S2B_MSIL2A_20191205T083229_N0213_R021_T36RTT_20191205T111147>


In [8]:
item = next(cat.get_items())
    
eo_item = extensions.eo.EOItemExt(item)      

for key in eo_item.item.assets.keys():

    if eo_item.item.assets[key].properties['eo:bands'][0]['common_name'] in ['swir16']:

         asset_swir16 = '/vsicurl/{}'.format(eo_item.item.assets[key].get_absolute_href())

    if eo_item.item.assets[key].properties['eo:bands'][0]['common_name'] in ['swir22']:

         asset_swir22 = '/vsicurl/{}'.format(eo_item.item.assets[key].get_absolute_href())

    if eo_item.item.assets[key].properties['eo:bands'][0]['common_name'] in ['nir']:

         asset_nir = '/vsicurl/{}'.format(eo_item.item.assets[key].get_absolute_href())

    if eo_item.item.assets[key].properties['eo:bands'][0]['common_name'] in ['red']:

         asset_red = '/vsicurl/{}'.format(eo_item.item.assets[key].get_absolute_href())

In [9]:
vrt = '{0}.vrt'.format(item.id)
    
ds = gdal.BuildVRT(vrt,
               [asset_red, asset_nir, asset_swir16, asset_swir22],
               srcNodata=0,
               xRes=10, 
               yRes=10,
               separate=True)

ds.FlushCache()

ds = None

del(ds)

In [10]:
tif = '{0}.tif'.format(item.id)
    
min_lon, min_lat, max_lon, max_lat = loads(aoi['value']).bounds

gdal.Translate(tif,
               vrt,
               outputType=gdal.GDT_Int16,
               projWin=[min_lon, max_lat, max_lon, min_lat],
               projWinSRS='EPSG:4326')

os.remove(vrt)

In [11]:
# remove STAC local catalog
shutil.rmtree('instac')

In [12]:
ds = gdal.Open(tif)
width = ds.RasterXSize
height = ds.RasterYSize

input_geotransform = ds.GetGeoTransform()
input_georef = ds.GetProjectionRef()

red = ds.GetRasterBand(1).ReadAsArray()
nir = ds.GetRasterBand(2).ReadAsArray()
swir16 = ds.GetRasterBand(3).ReadAsArray()
swir22 = ds.GetRasterBand(4).ReadAsArray()

ds = None

del(ds)

os.remove(tif)

In [13]:
nbr = np.zeros((height, width), dtype=np.uint)

nbr = 10000 * ((nir - swir22) / (nir + swir22))

swir22 = None

del(swir22)

ndvi = np.zeros((height, width), dtype=np.uint)

ndvi = 10000 * ((nir - red) / (nir + red))

red = None

del(red)

ndwi = np.zeros((height, width), dtype=np.uint)

ndwi = 10000 * ((nir - swir16) / (nir + swir16))

nir = swir16 = None

del(nir)

del(swir16)

In [14]:
# STAC output
default_bands = [{'name': 'NBR',
          'common_name': 'nbr'}, 
         {'name': 'NDVI',
          'common_name': 'ndvi'},
         {'name': 'NDWI',
          'common_name': 'ndwi'}]

In [15]:
catalog = Catalog(id='catalog-{}'.format(item.id),
                      description='Results')

catalog.clear_items()
catalog.clear_children()

<Catalog id=catalog-S2B_MSIL2A_20191205T083229_N0213_R021_T36RTT_20191205T111147>

In [16]:
item_name = 'INDEX_{}'.format(item.id)
    
    
item.properties.pop('eo:bands', None)
item.properties['eo:gsd'] = 10
item.properties['eo:platform'] = ['sentinel-2']
item.properties['eo:instrument'] = 'MSI'

result_item = Item(id=item_name,
                   geometry=item.geometry,
                   bbox=item.bbox,
                   datetime=item.datetime,
                   properties=item.properties)

result_item.common_metadata.set_gsd(10)

eo_item = extensions.eo.EOItemExt(result_item)

In [17]:
bands = []
    
for index, veg_index in enumerate(['NBR', 'NDVI', 'NDWI']):

    temp_name = '_{}_{}.tif'.format(veg_index, item.id)
    output_name = '{}_{}.tif'.format(veg_index, item.id)

    driver = gdal.GetDriverByName('GTiff')

    output = driver.Create(temp_name, 
                           width, 
                           height, 
                           1, 
                           gdal.GDT_Int16)

    output.SetGeoTransform(input_geotransform)
    output.SetProjection(input_georef)
    output.GetRasterBand(1).WriteArray(nbr),

    output.FlushCache()

    sleep(5)

    output = None

    del(output)

    os.makedirs(os.path.join('.', item_name),
                exist_ok=True)

    cog(temp_name, os.path.join('.', item_name, output_name))

    result_item.add_asset(key=veg_index.lower(),
                       asset=Asset(href='./{}'.format(output_name), 
                                   media_type=MediaType.GEOTIFF))

    asset = result_item.get_assets()[veg_index.lower()]                                   

    stac_band = extensions.eo.Band.create(name=veg_index.lower(), 
                                               common_name=default_bands[index]['common_name'],
                                               description=default_bands[index]['name'])
    bands.append(stac_band)

    eo_item.set_bands([stac_band], asset=asset)

eo_item.set_bands(bands)

eo_item.apply(bands)    

In [18]:
catalog.add_items([result_item])
    
catalog.normalize_and_save(root_href='./',
                            catalog_type=CatalogType.SELF_CONTAINED)
    